In [1]:
ls data

ms_hinds_buildings.geojson/     ms_hinds_locations.xlsx
ms_hinds_buildings.geojson.zip  ms_hinds_parcels.ndgeojson.gz
ms_hinds_locations.csv


In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
import duckdb

In [3]:
# Data Source 1
# CSV
# ms_hinds_locations = pd.read_csv("data/ms_hinds_locations.csv")
ms_hinds_locations = gpd.read_file("data/ms_hinds_locations.csv")
print(ms_hinds_locations.columns)
ms_hinds_locations.head(2)

Index(['source', 'parcel_id', 'parcel_lat', 'parcel_lon',
       'parcel_building_footprint', 'parcel_building_count', 'poi_lat',
       'poi_lon', 'secondary_lat', 'secondary_lon', 'lob_addr1', 'lob_lat',
       'lob_lon', 'lob_zipcode', 'f_ziploc', 'f_lat', 'f_lon', 'f_city',
       'primary_loc_id', 'f_addr1', 'f_unit', 'geometry'],
      dtype='object')


,source,parcel_id,parcel_lat,parcel_lon,parcel_building_footprint,parcel_building_count,poi_lat,poi_lon,secondary_lat,secondary_lon,...,lob_lon,lob_zipcode,f_ziploc,f_lat,f_lon,f_city,primary_loc_id,f_addr1,f_unit,geometry
0,POI,None,nan,nan,None,None,32.364986,-90.145943,nan,nan,...,-90.14568,39211-5404,39211,32.365143,-90.14568,Jackson,4950@@OLD CANTON@RD@&&39211,4950 OLD CANTON RD,nan,None
1,POI,None,nan,nan,None,None,32.389834,-90.110479,nan,nan,...,-90.11048,39211-2441,39211,32.389835,-90.11048,Jackson,6300@@OLD CANTON@RD@&&39211,6300 OLD CANTON RD,nan,None


In [4]:
# Data Source 2
# EXCEL
ms_hinds_locations_xlsx = pd.read_excel("data/ms_hinds_locations.xlsx")
print(ms_hinds_locations_xlsx.columns)
ms_hinds_locations_xlsx.head(2)

Index(['source', 'parcel_id', 'parcel_lat', 'parcel_lon',
       'parcel_building_footprint', 'parcel_building_count', 'poi_lat',
       'poi_lon', 'secondary_lat', 'secondary_lon', 'lob_addr1', 'lob_lat',
       'lob_lon', 'lob_zipcode', 'f_ziploc', 'f_lat', 'f_lon', 'f_city',
       'primary_loc_id', 'f_addr1', 'f_unit'],
      dtype='object')


,source,parcel_id,parcel_lat,parcel_lon,parcel_building_footprint,parcel_building_count,poi_lat,poi_lon,secondary_lat,secondary_lon,...,lob_lat,lob_lon,lob_zipcode,f_ziploc,f_lat,f_lon,f_city,primary_loc_id,f_addr1,f_unit
0,POI,None,NaN,NaN,None,None,32.364986,-90.145943,NaN,NaN,...,32.365143,-90.14568,39211-5404,39211,32.365143,-90.14568,Jackson,4950@@OLD CANTON@RD@&&39211,4950 OLD CANTON RD,NaN
1,POI,None,NaN,NaN,None,None,32.389834,-90.110479,NaN,NaN,...,32.389835,-90.11048,39211-2441,39211,32.389835,-90.11048,Jackson,6300@@OLD CANTON@RD@&&39211,6300 OLD CANTON RD,NaN


In [5]:
# Data Source 3
import gzip
with gzip.open("data/ms_hinds_parcels.ndgeojson.gz", 'rb') as f_in:
    g_parcels = gpd.read_file(f_in)
    g_parcels.head(2)

In [6]:
# Data Source 4
import zipfile
zf = zipfile.ZipFile("data/ms_hinds_buildings.geojson.zip") 
# df = pd.read_csv(zf)
# dfcsv = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings_join_table.csv")
# dfjson = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings.json")

ms_hinds_buildings = [gpd.read_file(zf.open(text_file.filename)) for text_file in zf.infolist() if text_file.filename.endswith('.json')]
df_ms_buildings = ms_hinds_buildings[0]

### COMPARE BUILDING DF WITH PARCELS AND EXCEL DFs 

In [ ]:
print(df_ms_buildings.columns)

In [ ]:
df_ms_buildings.head(1)

In [ ]:
df_ms_buildings['geometry'].loc(0)[0]

In [ ]:
ms_hinds_locations_xlsx.columns

In [ ]:
ms_hinds_locations.columns

In [ ]:
g_parcel_cols = g_parcels.columns.tolist()
print(g_parcel_cols)

In [7]:
list(set(ms_hinds_locations.columns).intersection(g_parcel_cols))

NameError: name 'g_parcel_cols' is not defined

In [ ]:
df_ms_buildings.columns

In [ ]:
# pd.set_option('display.max_rows', None)
# g_parcels.head(1).to_dict()
g_parcels['geoid'].head(10).to_dict()

In [ ]:
# 049650000478000000
ms_hinds_locations[ms_hinds_locations.parcel_id == '28049']

In [ ]:
ms_hinds_locations_xlsx.columns == ms_hinds_locations.columns

In [ ]:
ms_hinds_locations.describe()

In [ ]:
ms_hinds_locations.describe() == ms_hinds_locations.describe()

In [ ]:
ms_hinds_locations[['source', 'parcel_id', 'parcel_building_footprint', 'lob_addr1', 'lob_lat', 'lob_lon', 'lob_zipcode', 'f_lat', 'f_lon', 'f_city', 'f_addr1']]

### EXPLORE ZIP CODE in DF LOCATIONS

In [ ]:
duckdb.query("""
             WITH tmp AS (SELECT LEN(f_ziploc) as len FROM ms_hinds_locations)
             SELECT len, COUNT(1) as num_records
             FROM tmp
             GROUP BY len
             """).to_df()
# If raw coords are not available, f_ziploc = “28” (Mississippi statefips)
# So Mississippi don't exists at the monment 
# TODO: handle this later on

In [ ]:
duckdb.query("""
             WITH tmp AS (SELECT LEN(f_ziploc) as len FROM ms_hinds_locations_xlsx)
             SELECT len, COUNT(1) as num_records
             FROM tmp
             GROUP BY len
             """).to_df()

## EDA of Datasets

In [ ]:
# !pip install dtale
import dtale
# d = dtale.show(ms_hinds_locations)
d = dtale.show(ms_hinds_locations)
d.open_browser()

In [ ]:
# !pip install dtale
import dtale
# d = dtale.show(ms_hinds_locations)
d = dtale.show(df_ms_buildings)
d.open_browser()

In [ ]:
# !pip install dtale
import dtale
# d = dtale.show(ms_hinds_locations)
d = dtale.show(g_parcels)
d.open_browser()

In [ ]:
ms_hinds_locations[['f_lat', 'f_lon', 'f_city', 'f_addr1', 'f_ziploc']].head(20)

## BASIC CLEANING OF DATASETS

In [ ]:
# Basic Cleaning 

# Remove Hidden Characters in f_city
s = ms_hinds_locations['f_city']
printable = r'\w \!\"#\$%&\'\(\)\*\+,\-\./:;<»«؛،ـ\=>\?@\[\\\]\^_\`\{\|\}~'
s = s.str.replacer(r'[^{}]+'.format(printable), '')
# Replace any White spaces
s = s.str.replace(r'[ ]+', '')
# Convert to lower case
s = s.str.lower()


# f_addr1 Column
# Remove Hidden Characters in in f_addr1
s = df['f_addr1']
printable = r'\w \!\"#\$%&\'\(\)\*\+,\-\./:;<»«؛،ـ\=>\?@\[\\\]\^_\`\{\|\}~'
s = s.str.replacer(r'[^{}]+'.format(printable), '')
import unicodedata
# Normalize accent characters with normal characters Ascii utf-8 values 
s = s.apply(
	lambda x: unicodedata.normalize('NFKD', u'{}'.format(x)).encode('ASCII', 'ignore').decode('utf-8')
)

In [ ]:
ms_hinds_locations['f_ziploc']

In [ ]:
from uszipcode import SearchEngine
engine = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive)
zipcode = engine.by_zipcode(60007)
# zipcode.polygon
# help(zipcode)

In [ ]:
# to install: pip install duckdb
# mydf = pd.DataFrame({'a' : [1, 2, 3]})
# print(duckdb.query("""
#         SELECT CASE WHEN LEN(f_ziploc) = 5 THEN CAST(f_ziploc AS INT)
#             END AS f_ziploc 
#         FROM ms_hinds_locations""").to_df())
# zipcode.to_json()

In [ ]:
# !pip install dtale
import dtale
# d = dtale.show(ms_hinds_locations)
d = dtale.show(ms_hinds_locations[['source', 'parcel_id', 'parcel_building_footprint', 'lob_addr1', 'lob_lat', 'lob_lon', 'lob_zipcode', 'f_lat', 'f_lon', 'f_city', 'f_addr1', 'f_ziploc', 'f_unit']])
# d.open_browser()

### NEAREST POINTS IN GEO 

The nearest_points() function in shapely.ops calculates the nearest points in a pair of geometries.

READ MORE at https://shapely.readthedocs.io/en/stable/manual.html

In [ ]:
from shapely.ops import nearest_points

def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [ ]:
ms_hinds_locations.head(1)

In [ ]:
ms_hinds_locations.columns

In [ ]:
df_locations = ms_hinds_locations
df_locations.crs = 'epsg:4326'
df_locations = df_locations.to_crs({'init' :'epsg:4326'})

gdf_locations = gpd.GeoDataFrame(
    df_locations, geometry=gpd.points_from_xy(df_locations.f_lat, df_locations.f_lon, crs="EPSG:4326")
)
# df_locations = ms_hinds_locations[['parcel_id', 'f_lat', 'f_lon', 'f_city', 'f_addr1', 'f_ziploc']]

In [ ]:
df_ms_buildings = df_ms_buildings.to_crs(crs={'init': 'epsg:4326'})
df_ms_buildings['centroid'] = df_ms_buildings.centroid

In [ ]:
unary_union = gdf_locations.unary_union
# print(unary_union)

In [ ]:
gdf_locations.columns

In [ ]:
# unary_union = df_ms_buildings[['ed_bld_uuid', 'geometry']].unary_union
# print(unary_union)

In [ ]:
df_locations["nearest_roof"] = df_ms_buildings.apply(get_nearest_values, other_gdf=gdf_locations, point_column="centroid", axis=1)

In [ ]:
df_locations

In [ ]:
# Shapely’s nearest_points -function provides a nice and easy way of conducting the nearest neighbor analysis, 
# it can be quite slow. Using it also requires taking the unary union of the point dataset where all the Points 
# are merged into a single layer. This can be a really memory hungry and slow operation,
# that can cause problems with large point datasets

### FUTHER EXPLORATION OF DATASETS

In [8]:
# # !pip install sweetviz
# #importing sweetviz and visualizing our data
# import sweetviz as sv
# sweet_report = sv.analyze(ms_hinds_locations)
# # sweet_report.show_html('sweetviz_report.html')

In [ ]:
# !pip install openpyxl

In [ ]:
# from keplergl import KeplerGl
# map_1 = KeplerGl(data={"data_1": ms_hinds_locations[["f_lat", "f_lon"]]})
# map_1

In [ ]:
ms_hinds_locations.equals(ms_hinds_locations_xlsx)

In [ ]:
ms_hinds_locations_xlsx.describe()

In [ ]:
ms_hinds_locations.equals(ms_hinds_locations_xlsx)
# pd.concat([ms_hinds_locations,ms_hinds_locations_xlsx]).drop_duplicates(keep=False)

In [ ]:
g_parcels.head(2)
# df = geopandas.read_file(url)

In [ ]:
g_parcels['geometry'][0]

In [ ]:
ms_hinds_buildings = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings_join_table.csv")
ms_hinds_buildings = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings_join_table.csv")
ms_hinds_buildings.head(2)

In [ ]:
d = dtale.show(ms_hinds_buildings)
d.open_browser()

In [ ]:
dfcsv.head()

In [ ]:
dfjson.head()

In [ ]:
import zipfile
zf = zipfile.ZipFile("data/ms_hinds_buildings.geojson.zip") 
# df = pd.read_csv(zf)
# dfcsv = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings_join_table.csv")
# dfjson = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings.json")

ms_hinds_buildings = [gpd.read_file(zf.open(text_file.filename)) for text_file in zf.infolist() if text_file.filename.endswith('.json')]
df_ms_buildings = ms_hinds_buildings_df[0]

In [ ]:
df_ms_buildings.head(2)

In [ ]:
import dtale
d = dtale.show(df_ms_buildings)
d.open_browser()

In [ ]:
print(df_ms_buildings['geometry'][0])

In [ ]:
df_ms_buildings['geometry'][0]

In [ ]:
# !pip install gnssmapper
# !pip install unlzw3

In [ ]:
# import geopandas as gpd
# import pandas as pd
# import gnssmapper as gm

# pilot_log = gpd.read_file("zip://./pilot_study.geojson.zip", driver="GeoJSON")
# pilot_log.time = pilot_log.time.astype('datetime64')
# pilot_log.svid = pilot_log.svid.astype('string')

In [ ]:
# import geopandas as gpd
# import pandas as pd
# import gnssmapper as gm

# ms_hinds_buildings = gpd.read_file("zip://./data/ms_hinds_buildings.geojson.zip", driver="GeoJSON")

In [ ]:
# zipfile = "zip:///Users/chethan.umesha/Work/personal/autoetl/geotl/data/ms_hinds_buildings.geojson.zip"
# # gpd.read_file(zipfile)
# ms_hinds_buildings = gpd.read_file(zipfile, driver="GeoJSON")

In [ ]:
# !pip install pyogrio

In [ ]:
# os.environ['GDAL_ENABLE_DEPRECATED_DRIVER_GTM'] = 'NO'
# g_parcels = gpd.read_file("data/ms_hinds_buildings.geojson.zip")

In [ ]:
# g_parcels.head()